In [48]:
import importlib
from ucr_benchmark_template.modeling import train_noprop

importlib.reload(train_noprop)

from ucr_benchmark_template.modeling.train_noprop import (
    load_dataset, make_model, train, predict, evaluate, save_model
)

In [49]:
import optuna
import numpy as np
import pandas as pd
import os
import logging
import traceback

LOG_DIR = "."
LOG_FILE = os.path.join(LOG_DIR, "optuna_run.log")

# Ensure the folder exists
os.makedirs(LOG_DIR, exist_ok=True)

# Clear any existing handlers (important if Optuna or Jupyter adds one)
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Now configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler(LOG_FILE, mode='a', encoding='utf-8'),
        logging.StreamHandler()  # optional: also show logs in console
    ]
)

logging.info("=== Logging system initialized ===")

def objective(trial):
    try:
        datasets = [
        "ACSF1", "Adiac", "ArrowHead", "BME", "Beef", "BeetleFly", "BirdChicken", "CBF", "Car", "Chinatown",
        "ChlorineConcentration", "CinCECGTorso", "Coffee", "Computers","CricketX", "CricketY", "CricketZ", "Crop",
        "DiatomSizeReduction", "DistalPhalanxOutlineAgeGroup", "DistalPhalanxOutlineCorrect", "DistalPhalanxTW",
        "ECG200", "ECG5000", "ECGFiveDays", "EOGHorizontalSignal", "EOGVerticalSignal", "Earthquakes", "ElectricDevices",
        "EthanolLevel", "FaceAll", "FaceFour", "FacesUCR", "FiftyWords", "Fish", "FordA", "FordB", "FreezerRegularTrain",
        "FreezerSmallTrain", "GunPoint", "GunPointAgeSpan", "GunPointMaleVersusFemale", "GunPointOldVersusYoung", "Ham",
        "HandOutlines", "Haptics", "Herring", "HouseTwenty", "InlineSkate", "InsectEPGRegularTrain", "InsectEPGSmallTrain",
        "InsectWingbeatSound", "ItalyPowerDemand", "LargeKitchenAppliances", "Lightning2", "Lightning7", "Mallat", "Meat",
        "MedicalImages", "MiddlePhalanxOutlineAgeGroup", "MiddlePhalanxOutlineCorrect", "MiddlePhalanxTW",
        "MixedShapesRegularTrain", "MixedShapesSmallTrain", "MoteStrain", "NonInvasiveFetalECGThorax1",
        "NonInvasiveFetalECGThorax2", "OSULeaf", "OliveOil", "PhalangesOutlinesCorrect", "Phoneme", "PigAirwayPressure",
        "PigArtPressure", "PigCVP", "Plane", "PowerCons", "ProximalPhalanxOutlineAgeGroup", "ProximalPhalanxOutlineCorrect",
        "ProximalPhalanxTW", "RefrigerationDevices", "Rock", "ScreenType", "SemgHandGenderCh2", "SemgHandMovementCh2",
        "SemgHandSubjectCh2", "ShapeletSim", "ShapesAll", "SmallKitchenAppliances", "SmoothSubspace",
        "SonyAIBORobotSurface1", "SonyAIBORobotSurface2", "StarLightCurves", "Strawberry", "SwedishLeaf", "Symbols",
        "SyntheticControl", "ToeSegmentation1", "ToeSegmentation2", "Trace", "TwoLeadECG", "TwoPatterns", "UMD",
        "UWaveGestureLibraryAll", "UWaveGestureLibraryX", "UWaveGestureLibraryY", "UWaveGestureLibraryZ", "Wafer", "Wine",
        "WordSynonyms", "Worms", "WormsTwoClass", "Yoga"
    ]
    
        # --- Hyperparameters ---
        T = trial.suggest_int("T", 7, 7)
        emb_d = trial.suggest_categorical("embedding_dim", [0])
        eta = trial.suggest_float("eta", 0.2, 0.2, step=0.1)
        lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
        k_size = trial.suggest_categorical("k_size", [7])
        dropout = trial.suggest_float("dropout", 0.4, 0.4, step=0.1)
        
        n_blocks = trial.suggest_int("n_blocks", 2, 2)
        base_ch = trial.suggest_categorical("base_channels", [32, 64])
        channels = [base_ch * (2 ** i) for i in range(n_blocks)]
        
        n_layers = trial.suggest_int("n_layers", 3, 3)
        fc_layers = n_layers * [256]

        n_merged = trial.suggest_int("n_merged", 1, 1)
        base = trial.suggest_categorical("base_merged", [128])
        fc_merged = [max(base // (2 ** i), 32) for i in range(n_merged)]

        hp_str = (
            f"T={T}, emb={emb_d}, eta={eta}, lr={lr:.1e}, k={k_size}, dpout={dropout}, "
            f"ch={channels}, fc={fc_layers}, merg={fc_merged}"
        )

        logging.info(f"=== Starting Trial {trial.number} ===")
        logging.info(f"Hyperparameters: {hp_str}")
        
        accuracies = []
        for dataset in datasets:
            try:
                logging.info(f"Starting dataset: {dataset}")
                trainloader, testloader = load_dataset(dataset, batch_size=64)
                model = make_model(dataset, emb_d, T, k_size, dropout, channels, fc_layers, fc_merged)
                model, _ = train(model, trainloader, 600, T, eta, lr, 1e-5)
                y_true, y_pred = predict(model, testloader)
                acc = evaluate(y_true, y_pred)["accuracy"]

                accuracies.append(acc)
                logging.info(f"Finished {dataset} → Accuracy: {acc:.4f}")

            except Exception as e:
                error_msg = f"Error on dataset {dataset}: {e}"
                logging.error(error_msg)
                logging.error(traceback.format_exc())
                accuracies.append(np.nan)
                continue
        
        avg_acc = np.nanmean(accuracies)
        logging.info(f"Trial {trial.number} completed with average accuracy: {avg_acc:.4f}")
    
        # --- Save results to CSV ---
        csv_path = "optuna_results_overnight.csv"
    
        if not os.path.exists(csv_path):
            df = pd.DataFrame({"Dataset": datasets + ["AVERAGE"]})
        else:
            df = pd.read_csv(csv_path)
    
        trial_name = f"Trial_{trial.number+1}"
        run_data = accuracies + [avg_acc]
        df[trial_name] = run_data

        # Explicitly create string column for hyperparameters
        col_name = f"{trial_name}_params"
        if col_name not in df.columns:
            df[col_name] = pd.Series(dtype="object")
        df.loc[df["Dataset"] == "AVERAGE", col_name] = hp_str

        df.to_csv(csv_path, index=False)
    
        return avg_acc

    except Exception as e:
        logging.error(f"Trial {trial.number} failed with error: {e}")
        logging.error(traceback.format_exc())
        return np.nan

2025-10-10 10:23:23,821 [INFO] === Logging system initialized ===


In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=1)

[I 2025-10-10 10:23:24,122] A new study created in memory with name: no-name-80ff1428-5f17-4a92-ade4-90a5f62d8e5d
2025-10-10 10:23:24,124 [INFO] === Starting Trial 0 ===
2025-10-10 10:23:24,125 [INFO] Hyperparameters: T=7, emb=0, eta=0.2, lr=1.1e-04, k=7, dpout=0.4, ch=[32, 64], fc=[256, 256, 256], merg=[128]
2025-10-10 10:23:24,125 [INFO] Starting dataset: ACSF1
2025-10-10 10:24:12,394 [INFO] Finished ACSF1 → Accuracy: 0.5800
2025-10-10 10:24:12,395 [INFO] Starting dataset: Adiac
2025-10-10 10:27:03,868 [INFO] Finished Adiac → Accuracy: 0.7289
2025-10-10 10:27:03,871 [INFO] Starting dataset: ArrowHead
2025-10-10 10:27:29,972 [INFO] Finished ArrowHead → Accuracy: 0.7143
2025-10-10 10:27:29,976 [INFO] Starting dataset: BME
2025-10-10 10:27:55,960 [INFO] Finished BME → Accuracy: 0.9467
2025-10-10 10:27:55,961 [INFO] Starting dataset: Beef
2025-10-10 10:28:23,981 [INFO] Finished Beef → Accuracy: 0.9333
2025-10-10 10:28:23,982 [INFO] Starting dataset: BeetleFly
2025-10-10 10:28:49,471 [INF